In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,45688
2,Huelva,Confirmados PDIA 14 días,302
3,Huelva,Tasa PDIA 14 días,"58,849893797377085"
4,Huelva,Confirmados PDIA 7 días,163
5,Huelva,Tasa PDIA 7 dias,"31,763353274743263"
6,Huelva,Total Confirmados,45890
7,Huelva,Curados,44738
8,Huelva,Fallecidos,418


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  45688.0


/tmp/ipykernel_41211/1396922249.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  12585.0


/tmp/ipykernel_41211/2509482988.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

/tmp/ipykernel_41211/2690025379.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
/tmp/ipykernel_41211/2690025379.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
/tmp/ipykernel_41211/2690025379.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 6849 personas en los últimos 7 días 

Un positivo PCR cada 3425 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,45688.0,163.0,302.0,513170.0,31.763353,58.849894,39.0
Huelva-Costa,27029.0,79.0,156.0,289548.0,27.283905,53.877077,18.0
Condado-Campiña,14202.0,70.0,115.0,156231.0,44.805448,73.608951,17.0
Huelva (capital),12585.0,21.0,42.0,143837.0,14.599860,29.199719,8.0
Cartaya,2033.0,31.0,56.0,20083.0,154.359408,278.842802,6.0
Sierra de Huelva-Andévalo Central,4031.0,13.0,30.0,67391.0,19.290410,44.516330,5.0
Almonte,2305.0,10.0,20.0,24507.0,40.804668,81.609336,4.0
Valverde del Camino,857.0,9.0,20.0,12750.0,70.588235,156.862745,4.0
Isla Cristina,3136.0,7.0,11.0,21393.0,32.720983,51.418688,3.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Villarrasa,229.0,4.0,9.0,2211.0,180.913614,407.055631,2.0
Manzanilla,144.0,5.0,6.0,2118.0,236.071766,283.286119,3.0
Cartaya,2033.0,31.0,56.0,20083.0,154.359408,278.842802,6.0
Valverde del Camino,857.0,9.0,20.0,12750.0,70.588235,156.862745,4.0
Villalba del Alcor,502.0,5.0,5.0,3366.0,148.544266,148.544266,2.0
Palos de la Frontera,1044.0,11.0,14.0,11742.0,93.680804,119.230114,2.0
Moguer,1982.0,15.0,25.0,21867.0,68.596515,114.327525,1.0
Bonares,464.0,2.0,6.0,6060.0,33.003300,99.009901,0.0
Paterna del Campo,211.0,1.0,3.0,3457.0,28.926815,86.780445,0.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Lepe,3025.0,4.0,14.0,27880.0,14.347202,50.215208,0.0,0.285714
Ayamonte,2088.0,5.0,17.0,21104.0,23.692191,80.553450,0.0,0.294118
Paterna del Campo,211.0,1.0,3.0,3457.0,28.926815,86.780445,0.0,0.333333
Bonares,464.0,2.0,6.0,6060.0,33.003300,99.009901,0.0,0.333333
Sierra de Huelva-Andévalo Central,4031.0,13.0,30.0,67391.0,19.290410,44.516330,5.0,0.433333
Villarrasa,229.0,4.0,9.0,2211.0,180.913614,407.055631,2.0,0.444444
Gibraleón,947.0,4.0,9.0,12737.0,31.404569,70.660281,0.0,0.444444
Valverde del Camino,857.0,9.0,20.0,12750.0,70.588235,156.862745,4.0,0.450000
Almonte,2305.0,10.0,20.0,24507.0,40.804668,81.609336,4.0,0.500000
